# Artificial and Computational Intelligence Assignment 1

## Problem solving by Uninformed & Informed Search

List only the BITS (Name) of active contributors in this assignment:
1. _CHANDUPATLA ANIRUDH REDDY (2022DA04387)_
2. _ADITYA RUHELA             (2022DA04599)_
3. _MANIKANDAN S              (2022DA04396)_
4. ___________________
5. ___________________

Things to follow
1.	Use appropriate data structures to represent the graph and the path using python libraries
2.	Provide proper documentation
3.	Find the path and print it

Coding begins here

### 1.	Define the environment in the following block

List the PEAS decription of the problem here in this markdown block

The PEAS (Performance measure, Environment, Actuator and Sensor) for the case are as follows-

Performance Measure- The performance efficiency and effectiveness of the robot is measured on the basis of the ability to find the optimal path from the start position to the finish position, termed as ‘Goal’, while avoiding and minimizing the encounter with obstacles such as the Fire, bush and walls. The robot should be able to complete its task in a timely and efficient manner.

Environment- The environment of the rescue robot is the cave represented as a maze. The cave contains obstacles such as walls, bush and the fire. The task of the robot is to move through the maze from the starting point till the end point, that is the goal, and rescue the rabbit from the cave. While trying to navigate through the cave, the robot must find the optimal path, avoiding any penalties awarded for crossing the bush and the fire and at least a number of steps.

(Data Structure choosen to represent environment is Matrix)

Actuators- The actuators of the robot that rescues the rabbit are its movement capabilities. The robot can move in four directions, North, South, East and West. It uses these movements to navigate through the cave and find the optimal path to the finish position.

Sensors- The rescue robot has sensors that allows it to perceive the environment around it and helps it more in legal directions. The robot can use its sensors to detect obstacles such as the walls and penalty imposing cells of bush and fire. The sensor also allows the robot to determine its present position and helps it in moving towards the goal position.


Design the agent as PSA Agent(Problem Solving Agent)
Clear Initial data structures to define the graph and variable declarations is expected

States: The state is determined by both the agent location in NxM grid/maze and walkable path avoiding obstacles walls, fire and bush. number of total states will vary when considering obstacles. (here our grid/maze is 7x6)

Initial state: Any state can be designated as the initial state. (here we are given Initial state as (0,2))

Actions: agent can go LEFT, RIGHT, UP and DOWN

Transition model: can not pass through walls or border of maze.

Goal test: check if reached that cell which is way out of maze. (here we are given Goal state as (6,4))

Path cost: each transition from one cell to another cell is +3 also 
    
  * if cell has bush then +1 additional cost
        
  * if cell has fire then +5 additional cost
  
IMPORTATANT: Write distinct code block as below

In [1]:
#Code Block : Set Initial State (Must handle dynamic inputs)
start = (0, 2)

In [2]:
#Code Block : Set the matrix for transition & cost (as relevant for the given problem)
# we chose to encode the environment related data like possibilities of transitions from one cell to another, bushes and fire
# in binary format like fire 32 | bush 16 | right 8 | left 4 | up  2   | down 1
# examples:
# cell value = 25 its binary equivalent is 011001 so it means at that cell there is a bush and from here we can move down & right
# cell value = 36 its binary equivalent is 100100 so it means at that cell there is a fire and from here we can move left
# cell value = 15 its binary equivalent is 001111 so it means at that cell there is neither fire nor bush and from here we can move left, right, up & down
maze = [[25, 12, 5, 8, 12, 37],
        [3, 1, 11, 12, 21, 3],
        [3, 10, 7, 40, 14, 7],
        [11, 36, 3, 9, 12, 6],
        [3, 25, 14, 14, 13, 21],
        [2, 11, 12, 5, 3, 3],
        [8, 6, 8, 14, 7, 2]]

In [3]:
#Code Block : Write function to design the Transition Model/Successor function. Ideally this would be called while search algorithms are implemented
def can_walk_from_current(cell, to_new_position):
    return ((to_new_position == UP and cell & 2 == 2) or
            (to_new_position == DOWN and cell & 1 == 1) or
            (to_new_position == LEFT and cell & 4 == 4) or
            (to_new_position == RIGHT and cell & 8 == 8))


def does_not_cross_maze_border_from(maze,node_position):
    return (0 <= node_position[0] < (len(maze)) and
            0 <= node_position[1] < (len(maze[0])))

def manhattan_distance(p1, p2):
    return (abs(p1[0] - p2[0]) +
            abs(p1[1] - p2[1]))

bush_in = lambda cell: cell & 16 == 16
fire_at = lambda cell: cell & 32 == 32

LEFT, RIGHT, UP, DOWN = (0, -1), (0, 1), (-1, 0), (1, 0)

In [4]:
#Code block : Write fucntion to handle goal test (Must handle dynamic inputs). Ideally this would be called while search algorithms are implemented
def is_goal_state(state, goal= (6,4)):
    return state == goal

### 2.	Definition of Algorithm 1 (A* path finding algorithm)

In [35]:
#Code Block : Function for algorithm 1 implementation
class Node():
    def __init__(self, parent=None, position=None):
        self.parent = parent
        self.position = position

        self.g = 0
        self.h = 0
        self.f = 0

    def __eq__(self, other):
        return self.position == other.position

def A_STAR(maze, start, end):
    start_node = Node(None, start)
    start_node.g = start_node.h = start_node.f = 0
    end_node = Node(None, end)
    end_node.g = end_node.h = end_node.f = 0

    open_list = []
    closed_list = []
    open_list.append(start_node)
    while len(open_list) > 0:
        current_node = open_list[0]
        current_index = 0
        for index, item in enumerate(open_list):
            if item.f < current_node.f:
                current_node = item
                current_index = index

        open_list.pop(current_index)
        closed_list.append(current_node)
        print("visited this node:",current_node.position,"as it has least cost",current_node.f,"and added it to closed list:[",
              [ {"cell":e.position,"cost":e.f} for e in closed_list],']')
        if current_node == end_node:
            print('finally visited goal!')
            path = []
            current = current_node
            total_cost = current.g
            while current is not None:
                path.append(current.position)
                current = current.parent                
            return {'path_taken':'->'.join([str(e) for e in path[::-1]]), 'total_cost':total_cost}

        children = []
        for new_position in [LEFT, RIGHT, UP, DOWN]:
            node_position = (current_node.position[0] + new_position[0], current_node.position[1] + new_position[1])
            print("checking neighbor:",node_position)
            if does_not_cross_maze_border_from(maze,node_position): 
                cell = maze[current_node.position[0]][current_node.position[1]]
                if(can_walk_from_current(cell, new_position)):
                    children.append(Node(current_node, node_position))
                    print("neighbor ",node_position," is reachable so adding to children list")

        for child in children:
            if child not in closed_list:
                print("child",child.position,"is not visited before so checking it")
                child_cell = maze[child.position[0]][child.position[1]]
                step_cost = 3
                if(fire_at(child_cell)): step_cost += 5
                if(bush_in(child_cell)): step_cost += 1
                child.g = current_node.g + step_cost
                child.h = manhattan_distance(child.position, end_node.position)
                child.f = child.g + child.h
                print("child has f(n):",child.f,"g(n):",child.g,"h(n):",child.h)

                for open_node in open_list:
                    if child == open_node and child.g > open_node.g:
                        continue

                open_list.append(child)
                print("since this child",child.position
                      ,"has best g value out of existing open list nodes adding it to open_list/to_be_searched list" 
                      ,[ {"cell":e.position,"cost":e.f} for e in open_list])
        print("-"*10)

    return 'not able to find path'

Interpretation of the findings & Heuristics:

g(n) is the actual cost of the path from the start node to the current node. It is calculated by adding the distance from the start node to the parent node and the distance from the parent node to the current node.

h(n) is the estimated cost of the path from the current node to the goal node. It is calculated by using a heuristic function that approximates the distance based on some criteria, such as Euclidean distance, Manhattan distance, etc.

f(n) is the total cost of the path from the start node to the goal node through the current node. It is calculated by adding g(n) and h(n).

### 3.	Definition of Algorithm 2 (Random-restart hill climbing)

In [33]:
#Code Block : Function for algorithm 2 implementation
from random import randint
from queue import PriorityQueue

hill_state_value = []
goal_state = (6, 4)

for row in range(len(maze)):
    hill_state_value.append([])
    for column in range(len(maze[row])):
        hill_state_value[row].append(10 - manhattan_distance(goal_state, (row, column)))

def HILL_CLIMBING(maze, initial_state, iter_limit=50):
    current_state = initial_state
    walked_path = [current_state]
    variable_cost = 0
    while iter_limit != 0:
        cell = maze[current_state[0]][current_state[1]]
        neighbors = PriorityQueue()
        for to_new_position in [LEFT, RIGHT, UP, DOWN]:
            neighbor_position = current_state[0] + to_new_position[0], current_state[1] + to_new_position[1]
            print("checking neighbor:",neighbor_position)
            if (does_not_cross_maze_border_from(maze,neighbor_position)):
                if can_walk_from_current(cell, to_new_position) and not fire_at(cell) and neighbor_position not in walked_path:
                    neighbors.put_nowait((-hill_state_value[neighbor_position[0]][neighbor_position[1]], neighbor_position))
                    print("neighbor ",neighbor_position," is reachable so adding to neighbor list")
        if (neighbors.empty()): 
            print('no reachable neighbors have been found so returning failure')
            return None, None, None
        best_neighbor_value, best_neighbor_state = neighbors.get_nowait()
        best_cell = maze[best_neighbor_state[0]][best_neighbor_state[1]]
        print("picked neighbor cell",best_neighbor_state,'since it is closer to goal')
        variable_cost += 3
        if (fire_at(best_cell)): 
            print('encountered FIRE')
            variable_cost += 5
        if (bush_in(best_cell)): 
            print('encountered BUSH')
            variable_cost += 1
        walked_path += [best_neighbor_state]
        print('path walked so far',walked_path,'with cost',variable_cost)
        if (-best_neighbor_value <= hill_state_value[current_state[0]][current_state[1]]) or best_neighbor_state == goal_state:
            return best_neighbor_state, walked_path, variable_cost
        current_state = best_neighbor_state
        iter_limit -= 1


def RANDOM_RESTART_HILL_CLIMBING(maze, goal_state, iter_limit=50):
    searched_states = set()
    if (len(maze) > 0):
        vertical_length = len(maze[0]) - 1
        horizontal_length = len(maze) - 1
        while iter_limit != 0:
            random_initial_state = randint(0, horizontal_length), randint(0, vertical_length)
            while True:
                if (not random_initial_state in searched_states and 
                    not fire_at(maze[random_initial_state[0]][random_initial_state[1]])):
                    searched_states.add(random_initial_state)
                    break
                random_initial_state = randint(0, horizontal_length), randint(0, vertical_length)
            print('-'*10)
            print('selected random_initial_state:',random_initial_state)
            final_state, walked_path, final_cost = HILL_CLIMBING(maze, random_initial_state)
            if final_state == goal_state:
                print('found global maxima!')
                return {'path_taken':'->'.join([str(e) for e in walked_path]), 'total_cost':final_cost}
            else: print('got stuck at local maxima')
            iter_limit -= 1
            
    return 'not able to find path'

### DYNAMIC INPUT

IMPORTANT : Dynamic Input must be got in this section. Display the possible states to choose from:
This is applicable for all the relevent problems as mentioned in the question.

 (0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5),

 (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5),

 (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5),
 
 (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5),
 
 (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5),
 
 (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5),
 
 (6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5)

In [53]:
hill_state_value

[[0, 1, 2, 3, 4, 3],
 [1, 2, 3, 4, 5, 4],
 [2, 3, 4, 5, 6, 5],
 [3, 4, 5, 6, 7, 6],
 [4, 5, 6, 7, 8, 7],
 [5, 6, 7, 8, 9, 8],
 [6, 7, 8, 9, 10, 9]]

Interpretation: 

Cost function for Random Restart Hill Climbing is calculated by subtracting the Manhattan distance of each node to the goal from a numerical constant (here it was 10 but can be any) where the highest peak is the goal. the 3D graph of this hill_state_value forms a hill (its diagran is attached in document)

In [8]:
#Code Block : Function & call to get inputs (start/end state)
start = (0, 2)
goal = (6, 4)

### 4.	Calling the search algorithms
(For bidirectional search in below sections first part can be used as per Hint provided. Under second section other combinations as per Hint or your choice of 2 algorithms can be called .As an analyst suggest suitable approximation in the comparitive analysis section)

In [36]:
#Invoke algorithm 1 (Should Print the solution, path, cost etc., (As mentioned in the problem))
A_STAR(maze, start, goal)

visited this node: (0, 2) as it has least cost 0 and added it to closed list:[ [{'cell': (0, 2), 'cost': 0}] ]
checking neighbor: (0, 1)
neighbor  (0, 1)  is reachable so adding to children list
checking neighbor: (0, 3)
checking neighbor: (-1, 2)
checking neighbor: (1, 2)
neighbor  (1, 2)  is reachable so adding to children list
child (0, 1) is not visited before so checking it
child has f(n): 12 g(n): 3 h(n): 9
since this child (0, 1) has best g value out of existing open list nodes adding it to open_list/to_be_searched list [{'cell': (0, 1), 'cost': 12}]
child (1, 2) is not visited before so checking it
child has f(n): 10 g(n): 3 h(n): 7
since this child (1, 2) has best g value out of existing open list nodes adding it to open_list/to_be_searched list [{'cell': (0, 1), 'cost': 12}, {'cell': (1, 2), 'cost': 10}]
----------
visited this node: (1, 2) as it has least cost 10 and added it to closed list:[ [{'cell': (0, 2), 'cost': 0}, {'cell': (1, 2), 'cost': 10}] ]
checking neighbor: (1

{'path_taken': '(0, 2)->(1, 2)->(2, 2)->(3, 2)->(4, 2)->(4, 3)->(4, 4)->(5, 4)->(6, 4)',
 'total_cost': 24}

In [34]:
#Invoke algorithm 2 (Should Print the solution, path, cost etc., (As mentioned in the problem))
RANDOM_RESTART_HILL_CLIMBING(maze, goal)

----------
selected random_initial_state: (6, 1)
checking neighbor: (6, 0)
neighbor  (6, 0)  is reachable so adding to neighbor list
checking neighbor: (6, 2)
checking neighbor: (5, 1)
neighbor  (5, 1)  is reachable so adding to neighbor list
checking neighbor: (7, 1)
picked neighbor cell (5, 1) since it is closer to goal
path walked so far [(6, 1), (5, 1)] with cost 3
got stuck at local maxima
----------
selected random_initial_state: (1, 5)
checking neighbor: (1, 4)
checking neighbor: (1, 6)
checking neighbor: (0, 5)
neighbor  (0, 5)  is reachable so adding to neighbor list
checking neighbor: (2, 5)
neighbor  (2, 5)  is reachable so adding to neighbor list
picked neighbor cell (2, 5) since it is closer to goal
path walked so far [(1, 5), (2, 5)] with cost 3
checking neighbor: (2, 4)
neighbor  (2, 4)  is reachable so adding to neighbor list
checking neighbor: (2, 6)
checking neighbor: (1, 5)
checking neighbor: (3, 5)
neighbor  (3, 5)  is reachable so adding to neighbor list
picked nei

{'path_taken': '(5, 1)->(5, 2)->(5, 3)->(6, 3)->(6, 4)', 'total_cost': 12}

### 5.	Comparitive Analysis

### Time & Space complexity of A* algorithm

time complexity is O(b^d), where b is the branching factor and d is the length of the solution path. 

space complexity is also O(b^d), as it stores all generated nodes in memory

### Time & Space complexity of Random Restart Hill Climbing algorithm

time and space complexity of it depends on problem space, evaluation function, and number of restarts. 

worst case, the time complexity is O(∞), as it may never find the global optimum. 

or we can put limit on number of iterations(n_iter) so that it doesn't go to O(∞) and stops after that if global optimum is not found

space complexity is O(b), where b is the branching factor, as it discards old nodes

### 6.	Provide your comparitive analysis or findings in no more than 3 lines in below section

Comparison : 
* A* uses cost function f(n) = g(n){distance from start to current} + h(n){from current to goal here its Manhattan distance} whereas Random Restart Hill Climbing uses unit step distance from goal and its graph looks like hill
* A* gives same path everytime since the starting and end goal nodes are predefined whereas Random Restart Hill Climbing gives different path based on start cell selected randomly.
* Random Restart Hill Climbing would take more time to find a path as it might walk from the middle/part of the same route through which it had already passed in the previous iterations but was unable to find a solution. or sometimes it might start/restart at goal as well